In [1]:
import pandas as pd
import numpy as np 

In [2]:
data = pd.read_csv("spam.csv", encoding="latin-1")  # or "ISO-8859-1"
data.head()

,v1,v2,Unnamed: 2,Unnamed: 3,Unnamed: 4
0,ham,"Go until jurong point, crazy.. Available only ...",NaN,NaN,NaN
1,ham,Ok lar... Joking wif u oni...,NaN,NaN,NaN
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,NaN,NaN,NaN
3,ham,U dun say so early hor... U c already then say...,NaN,NaN,NaN
4,ham,"Nah I don't think he goes to usf, he lives aro...",NaN,NaN,NaN


In [3]:
import re
import unicodedata
def clean_text(text):
    text = unicodedata.normalize("NFKD", text)
    text = text.encode("ascii", "ignore").decode("utf-8", "ignore")
    text = re.sub(r'[•*|►▪●]', ' ', text)
    text = re.sub(r'[\r\n\t]', ' ', text)
    text = re.sub(r'[^a-zA-Z0-9\s]', ' ', text)
    text = text.lower()
    text = re.sub(r'\s+', ' ', text).strip()
    return text
data['v2'] = data['v2'].apply(clean_text)

In [4]:
x = data['v2'].iloc[:100].astype(str).tolist()
y = data['v1'].iloc[:100]
print(x,y)

['go until jurong point crazy available only in bugis n great world la e buffet cine there got amore wat', 'ok lar joking wif u oni', 'free entry in 2 a wkly comp to win fa cup final tkts 21st may 2005 text fa to 87121 to receive entry question std txt rate t c s apply 08452810075over18 s', 'u dun say so early hor u c already then say', 'nah i don t think he goes to usf he lives around here though', 'freemsg hey there darling it s been 3 week s now and no word back i d like some fun you up for it still tb ok xxx std chgs to send a1 50 to rcv', 'even my brother is not like to speak with me they treat me like aids patent', 'as per your request melle melle oru minnaminunginte nurungu vettam has been set as your callertune for all callers press 9 to copy your friends callertune', 'winner as a valued network customer you have been selected to receivea a900 prize reward to claim call 09061701461 claim code kl341 valid 12 hours only', 'had your mobile 11 months or more u r entitled to update 

In [5]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence  import pad_sequences
tokenizer = Tokenizer()
tokenizer.fit_on_texts(x)

sequence = tokenizer.texts_to_sequences(x)

vocab_size = len(tokenizer.word_index)
print(vocab_size)

import numpy as np
max_len = max(len(seq) for seq in sequence)
print("Maximum review length:", max_len)

751
Maximum review length: 60


In [6]:
max_len =60
x = pad_sequences(sequence , maxlen=max_len)

print("Sequences:\n", x)
print("Vocabulary Size:", vocab_size)


Sequences:
 [[  0   0   0 ...  76 251 131]
 [  0   0   0 ... 133   9 252]
 [  0   0   0 ... 140 263  14]
 ...
 [  0   0   0 ... 737   2 188]
 [  0   0   0 ...   7   9 749]
 [  0   0   0 ... 751   2 102]]
Vocabulary Size: 751


In [7]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
y  = le.fit_transform(y)

In [8]:
from sklearn.model_selection import train_test_split
x_train , x_test , y_train , y_test = train_test_split(x,y,test_size=0.2,random_state=2)

In [9]:
import keras_tuner as kt
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, SimpleRNN, LeakyReLU, PReLU, ELU, Activation, Embedding
from tensorflow.keras import optimizers, regularizers
from tensorflow.keras.preprocessing.sequence import pad_sequences
import tensorflow as tf

# ===== DEFINE CUSTOM HARD SWISH FUNCTION =====
def hard_swish(x):
    """Hard Swish activation function"""
    return x * tf.nn.relu6(x + 3) / 6

# ===== PAD YOUR SEQUENCES FIRST! =====
vocab_size = 3067
TIMESTEPS = 20  # Truncate from 720

# THIS IS CRITICAL - PAD BEFORE EVERYTHING ELSE!
# x_train = pad_sequences(x_train, maxlen=TIMESTEPS, padding='post', truncating='post')
# x_test = pad_sequences(x_test, maxlen=TIMESTEPS, padding='post', truncating='post')

# print(f"After padding - x_train shape: {x_train.shape}")
# print(f"After padding - x_test shape: {x_test.shape}")


# ===== HYPERMODEL CLASS WITH EMBEDDING =====
class RNNHyperModel(kt.HyperModel):
    def __init__(self, timesteps, vocab_size):
        self.timesteps = timesteps
        self.vocab_size = vocab_size
    
    def build(self, hp):
        model = Sequential()

        # Tune batch_size and epochs
        hp.Int('batch_size', 16, 128, step=16, default=32)
        hp.Int('epochs', 10, 100, step=10, default=50)

        # ===== EMBEDDING LAYER - CRITICAL FOR TEXT DATA =====
        embedding_dim = hp.Int('embedding_dim', 50, 256, step=50)
        model.add(Embedding(
            input_dim=self.vocab_size + 1,  # +1 for padding token
            output_dim=embedding_dim,
            input_length=self.timesteps,
            mask_zero=True
        ))

        # Tune number of RNN layers
        num_layers = hp.Int("num_rnn_layers", 1, 5, step=1)
        
        for i in range(num_layers):
            units = hp.Int(f"rnn_units_{i}", 16, 256, step=16)
            
            activation = hp.Choice(
                f"rnn_activation_{i}",
                ["tanh", "relu", "sigmoid", "leaky_relu", "prelu", "elu",
                 "selu", "swish", "gelu", "mish", "hard_swish",
                 "hard_sigmoid", "linear"]
            )
            
            dropout = hp.Float(f"rnn_dropout_{i}", 0.0, 0.5, step=0.05)
            recurrent_dropout = hp.Float(f"rnn_recurrent_dropout_{i}", 0.0, 0.3, step=0.05)
            l2_reg = hp.Float(f"rnn_l2_{i}", 0.0, 0.01, step=0.001)
            
            return_sequences = (i < num_layers - 1)
            
            # For custom activations, use 'linear' in SimpleRNN
            if activation in ["leaky_relu", "prelu", "elu", "swish", "gelu", "mish", "hard_swish"]:
                rnn_activation = "linear"
            else:
                rnn_activation = activation
            
            model.add(SimpleRNN(
                units=units,
                activation=rnn_activation,
                dropout=dropout,
                recurrent_dropout=recurrent_dropout,
                return_sequences=return_sequences,
                kernel_regularizer=regularizers.l2(l2_reg) if l2_reg > 0 else None,
                recurrent_regularizer=regularizers.l2(l2_reg) if l2_reg > 0 else None
            ))
            
            # Add custom activation layer if needed
            if activation == "leaky_relu":
                model.add(LeakyReLU())
            elif activation == "prelu":
                model.add(PReLU())
            elif activation == "elu":
                model.add(ELU())
            elif activation == "swish":
                model.add(Activation(tf.nn.swish))
            elif activation == "gelu":
                model.add(Activation(tf.nn.gelu))
            elif activation == "mish":
                model.add(Activation(lambda x: x * tf.math.tanh(tf.math.softplus(x))))
            elif activation == "hard_swish":
                model.add(Activation(hard_swish))  # Use custom hard_swish function

        # Optional Dense layers after RNN
        use_dense = hp.Boolean("use_dense_layers")
        if use_dense:
            num_dense = hp.Int("num_dense_layers", 1, 3, step=1)
            for i in range(num_dense):
                dense_units = hp.Int(f"dense_units_{i}", 16, 256, step=16)
                dense_activation = hp.Choice(
                    f"dense_activation_{i}",
                    ["sigmoid", "tanh", "relu", "leaky_relu", "prelu", "elu",
                     "selu", "swish", "gelu", "hard_sigmoid", "linear"]
                )
                l2_reg_dense = hp.Float(f"dense_l2_{i}", 0.0, 0.01, step=0.001)

                model.add(Dense(
                    units=dense_units,
                    kernel_regularizer=regularizers.l2(l2_reg_dense) if l2_reg_dense > 0 else None
                ))

                # Apply activation
                if dense_activation == "leaky_relu":
                    model.add(LeakyReLU())
                elif dense_activation == "prelu":
                    model.add(PReLU())
                elif dense_activation == "elu":
                    model.add(ELU())
                elif dense_activation == "swish":
                    model.add(Activation(tf.nn.swish))
                elif dense_activation == "gelu":
                    model.add(Activation(tf.nn.gelu))
                else:
                    model.add(Activation(dense_activation))

                dropout_rate = hp.Float(f"dense_dropout_{i}", 0.0, 0.5, step=0.05)
                if dropout_rate > 0:
                    model.add(Dropout(rate=dropout_rate))

        # Output layer
        model.add(Dense(1, activation="sigmoid"))

        # Tune optimizer
        optimizer_name = hp.Choice(
            "optimizer",
            ["sgd", "momentum", "nesterov", "adagrad", "adadelta",
             "rmsprop", "adam", "adamax", "nadam", "adamw"]
        )
        lr = hp.Float("learning_rate", 1e-5, 1e-2, sampling="log")

        if optimizer_name == "sgd":
            optimizer = optimizers.SGD(learning_rate=lr)
        elif optimizer_name == "momentum":
            optimizer = optimizers.SGD(learning_rate=lr, momentum=0.9)
        elif optimizer_name == "nesterov":
            optimizer = optimizers.SGD(learning_rate=lr, momentum=0.9, nesterov=True)
        elif optimizer_name == "adagrad":
            optimizer = optimizers.Adagrad(learning_rate=lr)
        elif optimizer_name == "adadelta":
            optimizer = optimizers.Adadelta(learning_rate=lr)
        elif optimizer_name == "rmsprop":
            optimizer = optimizers.RMSprop(learning_rate=lr)
        elif optimizer_name == "adam":
            optimizer = optimizers.Adam(learning_rate=lr)
        elif optimizer_name == "adamax":
            optimizer = optimizers.Adamax(learning_rate=lr)
        elif optimizer_name == "nadam":
            optimizer = optimizers.Nadam(learning_rate=lr)
        elif optimizer_name == "adamw":
            optimizer = optimizers.AdamW(learning_rate=lr)

        model.compile(
            optimizer=optimizer,
            loss="binary_crossentropy",
            metrics=["accuracy"]
        )
        return model


# ===== CUSTOM TUNER CLASS =====
class MyRNNTuner(kt.RandomSearch):
    def run_trial(self, trial, *args, **kwargs):
        kwargs['batch_size'] = trial.hyperparameters.get('batch_size')
        kwargs['epochs'] = trial.hyperparameters.get('epochs')
        return super(MyRNNTuner, self).run_trial(trial, *args, **kwargs)


# ===== CREATE HYPERMODEL INSTANCE =====
hypermodel = RNNHyperModel(timesteps=TIMESTEPS, vocab_size=vocab_size)

# ===== INITIALIZE TUNER =====
tuner = MyRNNTuner(
    hypermodel=hypermodel,
    objective="val_accuracy",
    max_trials=25,
    directory="rnn_tuner",
    project_name="full_rnn_tuning",
    overwrite=True
)

# ===== RUN HYPERPARAMETER SEARCH =====
print("Starting RNN hyperparameter search...")
print(f"Vocabulary size: {vocab_size}")
print(f"Sequence length (TIMESTEPS): {TIMESTEPS}")
print(f"x_train shape: {x_train.shape}")  # Should be (samples, 20)
print(f"Training samples: {len(x_train)}")
print(f"Test samples: {len(x_test)}")
print("-" * 60)

tuner.search(
    x_train,
    y_train,
    validation_data=(x_test, y_test),
    verbose=1
)

# ===== PRINT BEST HYPERPARAMETERS =====
best_hp = tuner.get_best_hyperparameters(1)[0]

print("\n" + "="*60)
print("BEST HYPERPARAMETERS")
print("="*60)

print("\n📝 Embedding Configuration:")
print(f"  Embedding Dimension: {best_hp.get('embedding_dim')}")

print("\n📊 Optimizer Configuration:")
print(f"  Optimizer: {best_hp.get('optimizer')}")
print(f"  Learning Rate: {best_hp.get('learning_rate'):.6f}")

print("\n🔄 RNN Architecture:")
print(f"  Number of RNN Layers: {best_hp.get('num_rnn_layers')}")

for i in range(best_hp.get("num_rnn_layers")):
    print(f"\n  RNN Layer {i+1}:")
    print(f"    Units: {best_hp.get(f'rnn_units_{i}')}")
    print(f"    Activation: {best_hp.get(f'rnn_activation_{i}')}")
    print(f"    Dropout: {best_hp.get(f'rnn_dropout_{i}'):.3f}")
    print(f"    Recurrent Dropout: {best_hp.get(f'rnn_recurrent_dropout_{i}'):.3f}")
    print(f"    L2 Regularization: {best_hp.get(f'rnn_l2_{i}'):.4f}")

print("\n🔗 Dense Layers:")
if best_hp.get("use_dense_layers"):
    print(f"  Number of Dense Layers: {best_hp.get('num_dense_layers')}")
    for i in range(best_hp.get("num_dense_layers")):
        print(f"\n  Dense Layer {i+1}:")
        print(f"    Units: {best_hp.get(f'dense_units_{i}')}")
        print(f"    Activation: {best_hp.get(f'dense_activation_{i}')}")
        print(f"    Dropout: {best_hp.get(f'dense_dropout_{i}'):.3f}")
        print(f"    L2 Regularization: {best_hp.get(f'dense_l2_{i}'):.4f}")
else:
    print("  No additional dense layers used")

print("\n⚙️ Training Configuration:")
print(f"  Batch Size: {best_hp.get('batch_size')}")
print(f"  Epochs: {best_hp.get('epochs')}")

# ===== BUILD AND EVALUATE BEST MODEL (FIXED) =====
print("\n" + "="*60)
print("BUILDING BEST MODEL FROM HYPERPARAMETERS")
print("="*60)

# Build model with best hyperparameters instead of loading
best_model = hypermodel.build(best_hp)
best_model.summary()

print("\n" + "="*60)
print("TRAINING BEST MODEL")
print("="*60)

# Train the best model
history = best_model.fit(
    x_train, 
    y_train,
    batch_size=best_hp.get('batch_size'),
    epochs=best_hp.get('epochs'),
    validation_data=(x_test, y_test),
    verbose=1
)

print("\n" + "="*60)
print("BEST MODEL EVALUATION")
print("="*60)
test_loss, test_accuracy = best_model.evaluate(x_test, y_test, verbose=0)
print(f"Test Loss: {test_loss:.4f}")
print(f"Test Accuracy: {test_accuracy:.4f}")

# Optional: Save the final model
best_model.save('best_rnn_model.h5')
print("\n✅ Best model saved as 'best_rnn_model.h5'")


Trial 25 Complete [00h 00m 42s]
val_accuracy: 0.10000000149011612

Best val_accuracy So Far: 0.949999988079071
Total elapsed time: 00h 13m 49s

BEST HYPERPARAMETERS

📝 Embedding Configuration:
  Embedding Dimension: 250

📊 Optimizer Configuration:
  Optimizer: adagrad
  Learning Rate: 0.000622

🔄 RNN Architecture:
  Number of RNN Layers: 3

  RNN Layer 1:
    Units: 240
    Activation: prelu
    Dropout: 0.400
    Recurrent Dropout: 0.050
    L2 Regularization: 0.0090

  RNN Layer 2:
    Units: 256
    Activation: linear
    Dropout: 0.450
    Recurrent Dropout: 0.000
    L2 Regularization: 0.0070

  RNN Layer 3:
    Units: 144
    Activation: selu
    Dropout: 0.100
    Recurrent Dropout: 0.250
    L2 Regularization: 0.0010

🔗 Dense Layers:
  Number of Dense Layers: 3

  Dense Layer 1:
    Units: 144
    Activation: leaky_relu
    Dropout: 0.400
    L2 Regularization: 0.0030

  Dense Layer 2:
    Units: 160
    Activation: hard_sigmoid
    Dropout: 0.400
    L2 Regularization: 0.0030


C:\Users\ma516\anaconda3\Lib\site-packages\keras\src\saving\saving_lib.py:797: UserWarning: Skipping variable loading for optimizer 'adagrad', because it has 2 variables whereas the saved optimizer has 21 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


ValueError: A total of 9 objects could not be loaded. Example error message for object <Embedding name=embedding, built=True>:

The shape of the target variable and the shape of the target value in `variable.assign(value)` must match. variable.shape=(752, 250), Received: value.shape=(3068, 250). Target variable: <Variable path=sequential/embedding/embeddings, shape=(752, 250), dtype=float32, value=[[-0.0316867   0.03240318 -0.03027389 ... -0.04839934 -0.04408095
  -0.00155345]
 [-0.02706999 -0.02510197 -0.00506289 ... -0.01302626 -0.03740447
  -0.04695771]
 [-0.01223681 -0.04356085  0.00931966 ...  0.00909568 -0.01757501
   0.01180089]
 ...
 [-0.00961591  0.04086367 -0.04490916 ... -0.01616105  0.03990659
   0.02817952]
 [ 0.04586116  0.00210671 -0.03703226 ...  0.04814288  0.04680561
   0.02801216]
 [-0.01439121 -0.02962635 -0.01505201 ... -0.00956391 -0.0035878
   0.04159129]]>

List of objects that could not be loaded:
[<Embedding name=embedding, built=True>, <SimpleRNNCell name=sim

In [10]:
# ===== PRINT BEST HYPERPARAMETERS =====
best_hp = tuner.get_best_hyperparameters(1)[0]

print("\n" + "="*60)
print("BEST HYPERPARAMETERS")
print("="*60)

print("\n📝 Embedding Configuration:")
print(f"  Embedding Dimension: {best_hp.get('embedding_dim')}")

print("\n📊 Optimizer Configuration:")
print(f"  Optimizer: {best_hp.get('optimizer')}")
print(f"  Learning Rate: {best_hp.get('learning_rate'):.6f}")

print("\n🔄 RNN Architecture:")
print(f"  Number of RNN Layers: {best_hp.get('num_rnn_layers')}")

for i in range(best_hp.get("num_rnn_layers")):
    print(f"\n  RNN Layer {i+1}:")
    print(f"    Units: {best_hp.get(f'rnn_units_{i}')}")
    print(f"    Activation: {best_hp.get(f'rnn_activation_{i}')}")
    print(f"    Dropout: {best_hp.get(f'rnn_dropout_{i}'):.3f}")
    print(f"    Recurrent Dropout: {best_hp.get(f'rnn_recurrent_dropout_{i}'):.3f}")
    print(f"    L2 Regularization: {best_hp.get(f'rnn_l2_{i}'):.4f}")

print("\n🔗 Dense Layers:")
if best_hp.get("use_dense_layers"):
    print(f"  Number of Dense Layers: {best_hp.get('num_dense_layers')}")
    for i in range(best_hp.get("num_dense_layers")):
        print(f"\n  Dense Layer {i+1}:")
        print(f"    Units: {best_hp.get(f'dense_units_{i}')}")
        print(f"    Activation: {best_hp.get(f'dense_activation_{i}')}")
        print(f"    Dropout: {best_hp.get(f'dense_dropout_{i}'):.3f}")
        print(f"    L2 Regularization: {best_hp.get(f'dense_l2_{i}'):.4f}")
else:
    print("  No additional dense layers used")

print("\n⚙️ Training Configuration:")
print(f"  Batch Size: {best_hp.get('batch_size')}")
print(f"  Epochs: {best_hp.get('epochs')}")

# ===== BUILD AND EVALUATE BEST MODEL (FIXED) =====
print("\n" + "="*60)
print("BUILDING BEST MODEL FROM HYPERPARAMETERS")
print("="*60)

# Build model with best hyperparameters instead of loading
best_model = hypermodel.build(best_hp)
best_model.summary()

print("\n" + "="*60)
print("TRAINING BEST MODEL")
print("="*60)

# Train the best model
history = best_model.fit(
    x_train, 
    y_train,
    batch_size=best_hp.get('batch_size'),
    epochs=best_hp.get('epochs'),
    validation_data=(x_test, y_test),
    verbose=1
)

print("\n" + "="*60)
print("BEST MODEL EVALUATION")
print("="*60)
test_loss, test_accuracy = best_model.evaluate(x_test, y_test, verbose=0)
print(f"Test Loss: {test_loss:.4f}")
print(f"Test Accuracy: {test_accuracy:.4f}")

# Optional: Save the final model
best_model.save('best_rnn_model.h5')
print("\n✅ Best model saved as 'best_rnn_model.h5'")



BEST HYPERPARAMETERS

📝 Embedding Configuration:
  Embedding Dimension: 250

📊 Optimizer Configuration:
  Optimizer: adagrad
  Learning Rate: 0.000622

🔄 RNN Architecture:
  Number of RNN Layers: 3

  RNN Layer 1:
    Units: 240
    Activation: prelu
    Dropout: 0.400
    Recurrent Dropout: 0.050
    L2 Regularization: 0.0090

  RNN Layer 2:
    Units: 256
    Activation: linear
    Dropout: 0.450
    Recurrent Dropout: 0.000
    L2 Regularization: 0.0070

  RNN Layer 3:
    Units: 144
    Activation: selu
    Dropout: 0.100
    Recurrent Dropout: 0.250
    L2 Regularization: 0.0010

🔗 Dense Layers:
  Number of Dense Layers: 3

  Dense Layer 1:
    Units: 144
    Activation: leaky_relu
    Dropout: 0.400
    L2 Regularization: 0.0030

  Dense Layer 2:
    Units: 160
    Activation: hard_sigmoid
    Dropout: 0.400
    L2 Regularization: 0.0030

  Dense Layer 3:
    Units: 64
    Activation: linear
    Dropout: 0.000
    L2 Regularization: 0.0070

⚙️ Training Configuration:
  Batch Siz

C:\Users\ma516\anaconda3\Lib\site-packages\keras\src\layers\core\embedding.py:97: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding_1 (Embedding)         │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ simple_rnn_3 (SimpleRNN)        │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ p_re_lu_1 (PReLU)               │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ simple_rnn_4 (SimpleRNN)        │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ simple_rnn_5 (SimpleRNN)        │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_4 (Dense)                 │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ leaky_re_lu_1 (LeakyReLU)       │ ?                      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_2 (Dropout)             │ ?                      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_5 (Dense)                 │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ activation_2 (Activation)       │ ?                      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_3 (Dropout)             │ ?                      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_6 (Dense)                 │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ activation_3 (Activation)       │ ?                      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_7 (Dense)                 │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)


TRAINING BEST MODEL
Epoch 1/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 12s 12s/step - accuracy: 0.7375 - loss: 10.2335 - val_accuracy: 0.9000 - val_loss: 10.2259
Epoch 2/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 387ms/step - accuracy: 0.5750 - loss: 10.4173 - val_accuracy: 0.9000 - val_loss: 10.2165
Epoch 3/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 458ms/step - accuracy: 0.7125 - loss: 10.2950 - val_accuracy: 0.9000 - val_loss: 10.2074
Epoch 4/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 470ms/step - accuracy: 0.6625 - loss: 10.3557 - val_accuracy: 0.9000 - val_loss: 10.1966
Epoch 5/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 446ms/step - accuracy: 0.6375 - loss: 10.3382 - val_accuracy: 0.9000 - val_loss: 10.1895
Epoch 6/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 453ms/step - accuracy: 0.6875 - loss: 10.3060 - val_accuracy: 0.9000 - val_loss: 10.1840
Epoch 7/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 402ms/step - accuracy: 0.6625 - loss: 10.3619 - val_accuracy: 0.9000 - val_loss: 10.1764
Epoch 8/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 425ms/step - accuracy: 0.6875 - loss:

Test Loss: 9.9961
Test Accuracy: 0.9000

✅ Best model saved as 'best_rnn_model.h5'


In [20]:
from sklearn.metrics import accuracy_score

# ===== 1. Predict probabilities =====
y_pred_prob = best_model.predict(x_test)  # shape: (num_samples, 1)

# ===== 2. Convert probabilities to class labels =====
y_pred_class = (y_pred_prob >= 0.5).astype(int)

# ===== 3. Print first 10 predictions =====
print("Predicted probabilities:\n", y_pred_prob[:10])
print("Predicted classes:\n", y_pred_class[:10])

# ===== 4. Calculate accuracy =====
accuracy = accuracy_score(y_test, y_pred_class)
print(f"\n✅ Test Accuracy: {accuracy:.4f}")


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 114ms/step
Predicted probabilities:
 [[0.15002075]
 [0.21642639]
 [0.19165233]
 [0.16032076]
 [0.15633705]
 [0.24003525]
 [0.19619317]
 [0.1932541 ]
 [0.1810203 ]
 [0.18481408]]
Predicted classes:
 [[0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]]

✅ Test Accuracy: 0.9000


In [22]:
from tensorflow.keras.models import load_model
from tensorflow.keras.activations import hard_swish

# Map custom function
custom_objects = {'hard_swish': hard_swish}

# Load model
best_model = load_model('best_rnn_model.h5', compile=False, custom_objects=custom_objects)


In [23]:
from tensorflow.keras.preprocessing.sequence import pad_sequences
import numpy as np

line = "go until jurong point crazy available only in bugis n great world la e buffet cine there got amore wat"
seq = tokenizer.texts_to_sequences([line])

TIMESTEPS = 20
seq_padded = pad_sequences(seq, maxlen=TIMESTEPS, padding='post', truncating='post')

# Predict
y_pred_prob = best_model.predict(seq_padded)
y_pred_class = int(y_pred_prob[0][0] >= 0.5)

print(f"Predicted probability: {y_pred_prob[0][0]:.4f}")
print(f"Predicted class: {y_pred_class}")


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 422ms/step
Predicted probability: 1.0000
Predicted class: 1
